In [ ]:
MV = ['F', 'WR', 'IW', 'S', 'SW', 'NC', 'CL']
# CV = ['CS', 'CW', 'P', 'L', 'SL', 'CP', 'D', 'RW']
CV = ['CS', 'CW', 'P', 'L', 'SL', 'CP', 'D', 'G', 'RW']

P = 1
M = 1

p0 = 20
n_max = 10

mv_past = {}
cv_past = {}

lim_dict = {
#     CV
    'CS': [1000, 6500],
    'CW': [1000, 5000],
    'P': [10, 16],
    'L': [800, 950],
    'SL': [60, 80],
    'CP': [60, 100],
    'D': [1.5, 1.8],
    'RW': [500, 2500],
    'G': [40, 55],
#     MV
    'F': [750, 1500],
    'WR': [0.25, 0.75],
    'IW': [450, 750],
    'S': [5, 14],
    'SW': [500, 950],
    'NC': [4, 9],
    'CL': [2000, 7000],
}

sp_dict = {
    'CS': 3300,
    'CW': 2700,
    'P': 15.0,
    'L': 850.0,
    'SL': 75.0,
    'CP': 70.0,
    'D': 1.7,
    'RW': 790,
    'G': 46.0
}
wsp_dict = {
    'CS': 0,
    'CW': 0,
    'P': 100,
    'L': 100,
    'SL': 20,
    'CP': 20,
    'D': 20,
    'RW': 0,
    'G': 100
}

cont = Cont(P=P, M=M, n_max=n_max, p0=p0)
for mv_i in MV:
    for i in range(M):
        cont.mv_dict[mv_i][i].STATUS = 1
        cont.mv_dict[mv_i][i].LOWER = lim_dict[mv_i][0]
        cont.mv_dict[mv_i][i].UPPER = lim_dict[mv_i][1]
    
for cv_i in CV:
    for i in range(P):
        if (cv_i == 'CW') or (cv_i == 'CS') or (cv_i == 'RW'):
            cont.cv_dict[cv_i][i].SPLO = None
            cont.cv_dict[cv_i][i].SPHI = None
        else:
            cont.cv_dict[cv_i][i].STATUS = 1
            cont.cv_dict[cv_i][i].SPLO = lim_dict[cv_i][0]
            cont.cv_dict[cv_i][i].SPHI = lim_dict[cv_i][1]
            cont.cv_dict[cv_i][i].WSP = wsp_dict[cv_i]
            cont.cv_dict[cv_i][i].SP = (lim_dict[cv_i][0] + lim_dict[cv_i][1])/2

for i in range(n_max):
    cont.cv_dict['RW'][P+i].value = cont.cv_dict['RW'][P+i].value - (cont.mv_dict['IW'][P+i].value+cont.mv_dict['SW'][P+i].value)        

cont.options.IMODE = 3

cont.CV_TYPE = 1

run_steps = 600
yp = []

for k in range(run_steps):
    if k == 0: 
        for mv_i in MV:
            mv_past[mv_i] = df[[mv_i]].iloc[p0+k:p0-n_max+k:-1].to_numpy().flatten()
        for cv_i in CV:
            cv_past[cv_i] = df[[cv_i]].iloc[p0+k-1:p0-n_max+k-1:-1].to_numpy().flatten()
    else:
        for mv_i in MV:
            mv_past[mv_i] = [val.value[0] for val in cont.mv_dict[mv_i][M-1:n_max+M-1]]
        for cv_i in CV:
            cv_past[cv_i] = [val.value[0] for val in cont.cv_dict[cv_i][P-1:n_max+P-1]]

    cont.update_model(mv_past, cv_past)

    cont.solve(disp=False)

    
    cont.get_results()
    
    cont.clear_data()